In [4]:
!pip install ftfy regex tqdm

In [6]:
!pip install --upgrade pip setuptools wheel

  Attempting uninstall: wheel
    Found existing installation: wheel 0.38.4
    Uninstalling wheel-0.38.4:
      Successfully uninstalled wheel-0.38.4


In [7]:
!pip install --no-cache-dir git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to c:\users\srikari\appdata\local\temp\pip-req-build-p57b5rsa
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git 'C:\Users\Srikari\AppData\Local\Temp\pip-req-build-p57b5rsa'
  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [14 lines of output]
  ERROR: Can not execute `setup.py` since setuptools failed to import in the build environment with exception:
  Traceback (most recent call last):
    File "<pip-setuptools-caller>", line 14, in <module>
    File "C:\Users\Srikari\anaconda3\Anaconda\lib\site-packages\setuptools\__init__.py", line 26, in <module>
      from .dist import Distribution
    File "C:\Users\Srikari\anaconda3\Anaconda\lib\site-packages\setuptools\dist.py", line 20, in <module>
      from . import (
    File "C:\Users\Srikari\anaconda3\Anaconda\lib\site-packages\setuptools\_entry_points.py", line 6, in <module>
      from jaraco.text import yield_lines
    File "C:\Users\Srikari\anaconda3\Anaconda\lib\site-packages\setuptools\_ve

In [8]:
!pip install transformers torchvision torch pillow

In [10]:
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import torch
import json
from tqdm import tqdm

# Load HuggingFace CLIP
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Load grouped questions JSON
with open("output/grouped_questions.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Predict answers using image similarity
for item in tqdm(data, desc=" Predicting answers"):
    images = [Image.open(item["question_image"]).convert("RGB")]
    for opt in item["option_images"]:
        images.append(Image.open(opt).convert("RGB"))

    inputs = clip_processor(images=images, return_tensors="pt", padding=True).to(device)

    with torch.no_grad():
        image_features = clip_model.get_image_features(**inputs)
        image_features = image_features / image_features.norm(p=2, dim=-1, keepdim=True)

    question_feat = image_features[0]  # First one is the question
    option_feats = image_features[1:]  # Rest are options

    sims = torch.matmul(option_feats, question_feat.unsqueeze(-1)).squeeze(-1)
    best_idx = sims.argmax().item()
    predicted_answer = item["option_images"][best_idx]

    item["answer"] = predicted_answer
    print(f"Answer: {predicted_answer}")

# Save to new JSON
with open("output/grouped_questions_with_answers.json", "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2)

print("\n All answers predicted and saved in: grouped_questions_with_answers.json")


config.json: 0.00B [00:00, ?B/s]

C:\Users\Srikari\anaconda3\Anaconda\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Srikari\.cache\huggingface\hub\models--openai--clip-vit-base-patch32. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regul

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

ValueError: Due to a serious vulnerability issue in `torch.load`, even with `weights_only=True`, we now require users to upgrade torch to at least v2.6 in order to use the function. This version restriction does not apply when loading files with safetensors.
See the vulnerability report here https://nvd.nist.gov/vuln/detail/CVE-2025-32434